# **PydanticAI-Agents-Tools-OpenAI**
It is a Python agent framework designed to make it less painful to build production grade applications with Generative AI.
https://github.com/pydantic/pydantic-ai




## **Why Use PydanticAI?**
- Provides structured data validation using **Pydantic**.
- Supports multiple **LLM providers**, but this guide focuses on **OpenAI’s models**.
- Ensures AI agents return **well-formatted and validated responses**.

### **Prerequisite:**
- You must have an **OpenAI API key**. Store it securely as an environment variable (`OPENAI_API_KEY`) to prevent exposure in the code.
- You must have an **https://exchangerate.host/access_key=""**. It provides monthly 100 free access with zero charge.

---

## **Setup**: Install Required Dependencies

Ensure you have the required libraries installed before running the code:

```python
%pip install pydantic-ai openai
```

In [ ]:
%pip install pydantic-ai openai

>  We import **BaseModel** from Pydantic to define our data schema.
**Agent** from PydanticAI to create our AI agent.
We also imported **Field** from Pydantic, which allows us to add extra metadata to model fields (like descriptions or value constraints), though it’s optional for basic usage.



In [117]:
import os
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai import Agent, RunContext


In [118]:
'''
Since Jupyter notebooks run an event loop in the background, you may need to use
'''
import nest_asyncio
nest_asyncio.apply()




> (In a real application, you’d load this from a secure config or environment, not hard-code it.)



In [119]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
os.environ["EXCHANGE_RATE_API_KEY"] = "<EXCHANGE_RATE_API_KEY>


## Step 1: Define a Pydantic Model for AI Agent’s Output
We need a structured model for AI-generated responses. In this example, our agent provides information about cities, so we define a Pydantic model:

### Expected Output Fields
```
city: Name of the city.
country: The country where the city is located.
reason: Why the city has that nickname.
famous_person_from_city: A well-known person from that city.
```



In [120]:
class CityInfo(BaseModel):
    """
    Represents structured information about a city.

    Attributes:
        city (str): Name of the city.
        country (str): Country where the city is located.
        reason (str): Reason behind the city's nickname or significance.
        famous_person_from_city (str): A well-known individual from the city.
    """
    city: str
    country: str
    reason: str
    famous_person_from_city: str


## Step 2: Create an AI Agent with OpenAI Model and Pydantic Schema
We create an instance of Agent from pydantic_ai, specifying:

- Model Name: "openai:gpt-4o" (you can replace it with another supported model).
- Result Type: CityInfo (the Pydantic model for structured output).

### Why This Matters?
- The framework validates responses against the schema.
- If the LLM response lacks fields or incorrect formatting, PydanticAI prompts the model to retry.

### Using System Prompts in PydanticAI:
- You can use static or dynamic prompts.
- Example: system_prompt="You are an AI assistant that provides brief city information."


In [121]:
"""
    Initializes an AI agent using GPT-4o via the pydantic_ai library.
    Agent: An instance of the Agent class configured to return CityInfo objects.
"""

# Choose an OpenAI model (GPT-4o in this example)
model_name = "openai:gpt-4o"

# Initialize the agent with the model and expected result type
agent = Agent(model_name, result_type=CityInfo)


In [125]:
"""
Demonstrates how to run a synchronous query to fetch city information.
agent (Agent): The AI agent used for generating city information.
"""
query = "The windy city in the US of A."
result = agent.run_sync(query)


In [126]:
'''
When you run this, you should see that:
result:  is an AgentRunResult (a wrapper class that PydanticAI uses to encapsulate the outcome of the run, including metadata like tokens used or the message history).
result.data:  is an instance of our CityInfo Pydantic model (or whatever type you specified as result_type).
Printing result.data will display the field values that the model provided.
'''
print(type(result))        # What type is result?
print(type(result.data))   # What type is the data attribute?
print(result.data)         # The structured data output


<class 'pydantic_ai.agent.AgentRunResult'>
<class '__main__.CityInfo'>
city='Chicago' country='United States' reason="Known as 'The Windy City' due to its breezes off Lake Michigan and historically because of its reputation for verbose politicians." famous_person_from_city='Walt Disney'


In [127]:
city_info = result.data  # This is a CityInfo instance
print(city_info.city)            # Access the city field
print(city_info.famous_person_from_city)  # Access the famous person field

# Convert the result to a dictionary (e.g., to return as JSON in an API)
print(city_info.dict())


Chicago
Walt Disney
{'city': 'Chicago', 'country': 'United States', 'reason': "Known as 'The Windy City' due to its breezes off Lake Michigan and historically because of its reputation for verbose politicians.", 'famous_person_from_city': 'Walt Disney'}


<ipython-input-127-366fff44359c>:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(city_info.dict())


# Using PydanticAI Tools for AI Agents
We extend our AI agent with tools for additional functionality.


> PydanticAI Agent with Math and Currency Conversion Tools



## Tool 1: Math Tool (Performing Arithmetic Operations)

### We define two models:

- MathInput: Takes two numbers and an operation (add, subtract, multiply, divide).
- MathOutput: Stores the result.
- To ensure correct inputs, we use Python Literals or Enums for the operation field.

In [128]:
'''
Tool 1: Math Calculation Tool

'''

from pydantic import BaseModel
from typing import Literal

class MathInput(BaseModel):
    """
      Represents the input format for performing mathematical operations.

      Attributes:
          x (float): First operand.
          y (float): Second operand.
          operation (Literal[str]): Type of operation ('add', 'subtract',
                                    'multiply', or 'divide').
    """
    x: float
    y: float
    operation: Literal['add', 'subtract', 'multiply', 'divide']

class MathOutput(BaseModel):
    """
      Represents the output format of a math calculation.

      Attributes:
          result (float): The numerical result of the performed operation.
    """
    result: float


In [129]:

def calculate(ctx: RunContext, x: float, y: float, operation: str) -> MathOutput:
    """
    Perform basic arithmetic operations based on the specified operation.

    Args:
        ctx (RunContext): Execution context (provided automatically by pydantic_ai).
        x (float): First operand.
        y (float): Second operand.
        operation (str): The operation to perform, e.g. 'add', 'subtract',
                         'multiply', or 'divide'. Various synonyms are also accepted.

    Raises:
        ValueError: If the operation is invalid or if attempting to divide by zero.

    Returns:
        MathOutput: A MathOutput instance containing the computed result.
    """

    print(f"operation: {operation}")
    # Expanded operation mapping to handle more variations
    operation_mapping = {
        "plus": "add", "add": "add",
        "minus": "subtract", "subtract": "subtract",
        "times": "multiply", "multiplied": "multiply", "multiply": "multiply",
        "divided": "divide", "division": "divide", "divide": "divide",
        "sum": "add", "difference": "subtract", "product": "multiply", "quotient": "divide"
    }

    # Normalize operation (if invalid, return an error)
    operation = operation_mapping.get(operation.lower())
    if operation is None:
        raise ValueError("Invalid operation. Use: 'add', 'subtract', 'multiply', 'divide'.")


    # Perform calculation
    if operation == 'add':
        result_value = x + y
    elif operation == 'subtract':
        result_value = x - y
    elif operation == 'multiply':
        result_value = x * y
    elif operation == 'divide':
        if y == 0:
            raise ValueError("Cannot divide by zero.")
        result_value = x / y
    else:
        raise ValueError("Unsupported operation. Choose from 'add', 'subtract', 'multiply', 'divide'.")

    return MathOutput(result=result_value)


## Tool 2: Currency Conversion Tool
- This tool converts an amount from one currency to another using ExchangeRate.host.

### Pydantic Models for Currency Tool
- ConversionInput:
```
amount (float): The value to convert.
from_currency (str): Currency code (e.g., "USD").
to_currency (str): Target currency (e.g., "EUR").
```

- ConversionOutput:
```
converted_amount (float): The final amount.
rate (float): The exchange rate applied.
```


In [130]:
class ConversionInput(BaseModel):
    """
      Represents the input schema for performing currency conversions.

      Attributes:
          amount (float): The amount of money to convert.
          from_currency (str): Source currency code (e.g., 'USD').
          to_currency (str): Target currency code (e.g., 'EUR').
    """
    amount: float
    from_currency: str
    to_currency: str

class ConversionOutput(BaseModel):
    """
      Represents the output schema for a currency conversion.

      Attributes:
          converted_amount (float): The converted monetary amount.
          rate (float): The exchange rate applied for the conversion.
    """
    converted_amount: float
    rate: float


In [131]:
'''
Note: In a real application, you should handle API errors and possibly cache results, and other posible scenaios. This is just a demo example
'''
import requests

def exchange_currency(ctx: RunContext, amount: float, from_currency: str, to_currency: str) -> ConversionOutput:
    """Convert a monetary amount from one currency to another using real-time exchange rates.

    Args:
        ctx (RunContext): PydanticAI context.
        amount (float): Amount to convert.
        from_currency (str): Source currency (ISO 4217).
        to_currency (str): Target currency (ISO 4217).

    Returns:
        ConversionOutput: Converted amount and exchange rate.

    Note: This function fetches live exchange rates from an API.
          Consider caching results to reduce API calls.
    """

    API_KEY = "EXCHANGE RATE API KEY"
    BASE_URL = "https://api.exchangerate.host/convert"

    # Construct API request URL
    params = {
        "access_key": API_KEY,
        "from": from_currency.upper(),
        "to": to_currency.upper(),
        "amount": amount
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=5)  # Add timeout for reliability
        response.raise_for_status()  # Raise exception for HTTP errors
        data = response.json()

        # Extract required data
        converted = data.get("result")
        rate = data.get("info", {}).get("quote") if data.get("info") else None

        if converted is None or rate is None:
            raise ValueError(f"Failed to retrieve exchange rate from {from_currency} to {to_currency}.")

        print(f"✅ Successfully converted {amount} {from_currency} to {converted} {to_currency} at rate {rate}")

        # Return structured output
        return ConversionOutput(converted_amount=converted, rate=rate)

    except requests.exceptions.RequestException as e:
        raise ValueError(f"❌ API request failed: {e}")

## Integrating Tools into the AI Agent
PydanticAI allows seamless integration of tools, ensuring structured inputs and outputs.

In [132]:
# Initialize the agent with a model (e.g., OpenAI GPT-4o etc).
agent = Agent("openai:gpt-4o", tools=[calculate, exchange_currency])

In [133]:
"""
    (Optional)  Set a system prompt to guide the agent's behavior for calculations and currency conversions.
"""

agent.system_prompt = (
    "You are a helpful AI that can perform arithmetic operations using the 'calculate' tool "
    "and convert currencies using the 'exchange_currency' tool. "

    "For calculations, always format your tool calls using one of these operations: 'add', 'subtract', 'multiply', or 'divide'. "
    "Instead of saying 'minus', use 'subtract', and instead of 'times', use 'multiply'. "
    "If a user provides a multi-step arithmetic expression, break it down step by step and compute each part separately before proceeding. "

    "For currency conversions, ensure that both currencies are valid ISO 4217 currency codes (e.g., 'USD', 'EUR', 'JPY'). "
    "Always return the conversion result in numerical form along with the exchange rate. "

    "⚠️ Do not make assumptions about exchange rates; always use the 'exchange_currency' tool to get real-time data. "
    "⚠️ Do not attempt to perform arithmetic manually; always call the 'calculate' tool for computations. "
    "⚠️ If a request does not match a valid calculation or currency conversion, do not make up information—return an appropriate error response. "
)





In [111]:
# Test operation mapping and calculation
# print(calculate(None, 15, 4, "minus"))  # Should return MathOutput(result=11)
# print(calculate(None, 15, 4, "subtract"))  # Should return MathOutput(result=11)
# print(calculate(None, 15, 4, "times"))  # Should return MathOutput(result=60)


## Example Usage
- **Putting It All Together**
- The AI agent can now perform math calculations and convert currencies dynamically.
- PydanticAI ensures correct parameter passing and validates tool responses.


In [134]:
"""
Demonstrates how the agent can respond to a query involving arithmetic calculations.
"""
result = agent.run_sync("What is 15 minus 4 and multiple 100 ?")
print(result)

operation: subtract
operation: multiply
AgentRunResult(data='15 minus 4 is 11, and when you multiply 11 by 100, the result is 1100.')


In [135]:
"""
Demonstrates how the agent can respond to a query involving currency conversion.
"""
result = agent.run_sync("convert 100k usd to inr")
print(result)

✅ Successfully converted 100000.0 USD to 8694475.3 INR at rate 86.944753
AgentRunResult(data='100,000 USD is approximately 8,694,475.30 INR, based on the current exchange rate of 1 USD = 86.944753 INR.')


In [136]:
'''
Demonstrates a combined query that requires both currency conversion and arithmetic.
'''
result = agent.run_sync("Convert 50 USD to EUR and then add the result to 100.")
print(result)


✅ Successfully converted 50.0 USD to 45.94575 EUR at rate 0.918915
operation: add
AgentRunResult(data='Converting 50 USD to EUR gives approximately 45.95 EUR. Adding this to 100 results in 145.95.')


# **Conclusion**

In this tutorial, we:
- ✅ Built a PydanticAI-powered AI agent.
- ✅ Defined structured Pydantic schemas for validation.
- ✅ Integrated math & currency conversion tools into the agent.
- ✅ Ensured LLM responses match predefined formats automatically.
- ✅ This is a very basic fundamental implementation and can be extended with more agents, optimizations, custom tools, state managemnt.

By using PydanticAI, AI agents become more reliable, structured, and error-resistant.
And, special thanks to pydantic-ai documentation and openai documentations.